In [ ]:
# %%
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:



# %%

"""
Main Entry Point
"""
from pathlib import Path

import pandas as pd  # type: ignore

import cdutils.pkey_sqlite  # type: ignore
import cdutils.filtering  # type: ignore
import cdutils.input_cleansing  # type: ignore
import cdutils.cmo_append  # type: ignore
import src.add_fields
import src.core_transform
import src.output_to_excel
from src._version import __version__
import src.output_to_excel_multiple_sheets
import cdutils.distribution  # type: ignore
from datetime import datetime
from dateutil.relativedelta import relativedelta

# # Ensure BASE_PATH for output
# try:
#     BASE_PATH
# except NameError:
#     BASE_PATH = Path('.')
# (BASE_PATH / "output").mkdir(parents=True, exist_ok=True)

# %%

# Get staging data from the daily deposit update. View dev section of documentation for more detail
INPUT_PATH = Path(r"\\00-da1\Home\Share\Data & Analytics Initiatives\Project Management\Data_Analytics\Daily_Deposit_Update\Production\output\DailyDeposit_staging.xlsx")
data = pd.read_excel(INPUT_PATH)

# Add portfolio key
data = cdutils.pkey_sqlite.add_pkey(data)

# Add int rate
data = src.add_fields.add_noteintrate(data)

# Custom list of minors (Business Deposits)
minors = [
    'CK24',  # 1st Business Checking
    'CK12',  # Business Checking
    'CK25',  # Simple Business Checking
    'CK30',  # Business Elite Money Market
    'CK19',  # Business Money Market
    'CK22',  # Business Premium Plus MoneyMkt
    'CK23',  # Premium Business Checking
    'CK40',  # Community Assoc Reserve
    'CD67',  # Commercial Negotiated Rate
    'CD01',  # 1 Month Business CD
    'CD07',  # 3 Month Business CD
    'CD17',  # 6 Month Business CD
    'CD31',  # 1 Year Business CD
    'CD35',  # 1 Year Business CD
    'CD37',  # 18 Month Business CD
    'CD38',  # 2 Year Business CD
    'CD50',  # 3 Year Business CD
    'CD53',  # 4 Year Business CD
    'CD59',  # 5 Year Business CD
    'CD76',  # 9 Month Business CD
    'CD84',  # 15 Month Business CD
    'CD95',  # Business <12 Month Simple CD
    'CD96',  # Business >12 Month Simple CD
    'CK28',  # Investment Business Checking
    'CK33',  # Specialty Business Checking
    'CK34',  # ICS Shadow - Business - Demand
    'SV06',  # Business Select High Yield
    'CK13',
    'CK15',
    'CK41'
]

# Filter to only business deposit accounts
data = cdutils.filtering.filter_to_business_deposits(data, minors)

# Add CMO
data = cdutils.cmo_append.append_cmo(data)

data_schema = {
    'noteintrate': float
}
data = cdutils.input_cleansing.enforce_schema(data, data_schema).copy()

# Exclude BCSB internal accounts
data = data[~data['ownersortname'].str.contains('BRISTOL COUNTY SAVINGS', case=False, na=False)].copy()




In [ ]:
# %%
# %%
ASSETS_PATH = Path('./assets')

files = [f for f in ASSETS_PATH.iterdir() if f.is_file()]

assert len(files) == 1, f"Expected exactly 1 file in {ASSETS_PATH}, found {len(files)}."

file = files[0]
assert file.suffix == '.csv', f"Expected a csv file"

xaa_data = pd.read_csv(file)

# %%
# xaa_data.info()

#

# %%




# # %%
# xaa_data['Analyzed Charges (Pre-ECR)'] = xaa_data['Analyzed Charges (Pre-ECR)'].str.replace('[\$,]','',regex=True)
# xaa_data['Combined Result for Settlement Period (Post-ECR + Fee-Based Total)'] = xaa_data['Combined Result for Settlement Period (Post-ECR + Fee-Based Total)'].str.replace('[\$,]','',regex=True)

# # Rename to match schema from earlier
# xaa_data = xaa_data.rename(columns={
#     'Analyzed Charges (Pre-ECR)':'Analyzed Charges',
#     'Combined Result for Settlement Period (Post-ECR)':'Combined Result for Settlement Period'
# })
# # fix csv formatting of float fields
# cols_to_adjust = ['Analyzed Charges','Combined Result for Settlement Period']

# for col in cols_to_adjust:
#     xaa_data[col] = xaa_data[col].str.replace(r'[$,]','', regex=True).astype(float)

# %%
xaa_schema = {
    'Analyzed Charges':'float',
    'Combined Result for Settlement Period':'float',
    'Earnings Credit Rate':'float',
    'Debit Account Number':'str'
}
xaa_data = cdutils.input_cleansing.enforce_schema(xaa_data, xaa_schema)

xaa_data['Analyzed Result Month-to-Date'] = xaa_data['Analyzed Result Month-to-Date'] * -1
xaa_data['Combined Result for Settlement Period'] = xaa_data['Combined Result for Settlement Period'] * -1
xaa_data['Analyzed Result Month-to-Date'] = xaa_data['Analyzed Result Month-to-Date'].mask(xaa_data['Analyzed Result Month-to-Date'] == 0, 0.0)
xaa_data['Combined Result for Settlement Period'] = xaa_data['Combined Result for Settlement Period'].mask(xaa_data['Combined Result for Settlement Period'] == 0, 0.0)
# %%

from datetime import datetime, timedelta


# %%

# ---------- Summarizer that matches your rules ----------
from typing import Optional, Union

def summarize_xaa_for_latest_and_ttm(
    xaa_df: pd.DataFrame,
    *,
    date_col: str = "Cycle End Date",
    account_col: str = "Debit Account Number",
    charges_col: str = "Analyzed Charges",
    result_col: str = "Combined Result for Settlement Period",
    ecr_col: str = "Earnings Credit Rate",
    target_month: Optional[Union[str, pd.Period, pd.Timestamp]] = None,
    today: Optional[pd.Timestamp] = None,
) -> pd.DataFrame:
    """
    Build per-account summary using:
      - Latest month: previous month-end on/before 'today', unless 'target_month' is provided (e.g. '2025-08' -> 2025-08-31)
      - Trailing 12 months: dates within [today - 12 months, today], inclusive

    Returns one row per account with:
      account_col,
      Latest_Month_Analyzed_Charges, Latest_Month_Combined_Result, Latest_Month_ECR,
      Trailing_12M_Analyzed_Charges, Trailing_12M_Combined_Result, Trailing_12M_Avg_ECR,
      Primary_Officer_Name_XAA, Secondary_Officer_Name_XAA, Treasury_Officer_Name_XAA
    """
    df = xaa_df.copy()
    df[date_col] = pd.to_datetime(df[date_col]).dt.normalize()
    if today is None:
        today = pd.Timestamp.today().normalize()

    # Determine target month-end date
    if target_month is not None:
        if isinstance(target_month, pd.Timestamp):
            target_period = target_month.to_period("M")
        elif isinstance(target_month, pd.Period):
            target_period = target_month.asfreq("M")
        else:
            target_period = pd.Period(str(target_month), freq="M")
        target_eom = target_period.to_timestamp(how="end").normalize()
    else:
        # Most recent end-of-month on or before 'today'
        target_eom = today if today.is_month_end else (today - pd.offsets.MonthEnd(1)).normalize()

    # Masks
    target_mask = df[date_col].eq(target_eom)
    ttm_start = (today - relativedelta(months=12)).normalize()
    trailing_mask = (df[date_col] >= ttm_start) & (df[date_col] <= today)

    # Coerce numerics robustly
    def _coerce_numeric(s: pd.Series) -> pd.Series:
        if s.dtype == "O":
            s = (s.astype(str)
                   .str.replace(r"[$,]", "", regex=True)
                   .replace({"": None}))
        return pd.to_numeric(s, errors="coerce")

    for col in (charges_col, result_col, ecr_col):
        if col in df.columns:
            df[col] = _coerce_numeric(df[col])

    # Officer names per account
    base = (df.groupby(account_col, as_index=False)
              .agg(Primary_Officer_Name_XAA=("Primary Officer Name", "first"),
                   Secondary_Officer_Name_XAA=("Secondary Officer Name", "first"),
                   Treasury_Officer_Name_XAA=("Treasury Officer Name", "first")))

    # Target month
    target = (df.loc[target_mask]
                .groupby(account_col, as_index=False)
                .agg(Latest_Month_Analyzed_Charges=(charges_col, "sum"),
                     Latest_Month_Combined_Result=(result_col, "sum"),
                     Latest_Month_ECR=(ecr_col, "mean")))

    # Trailing 12 months
    trailing = (df.loc[trailing_mask]
                  .groupby(account_col, as_index=False)
                  .agg(Trailing_12M_Analyzed_Charges=(charges_col, "sum"),
                       Trailing_12M_Combined_Result=(result_col, "sum"),
                       Trailing_12M_Avg_ECR=(ecr_col, "mean")))

    out = (base
           .merge(target, on=account_col, how="left")
           .merge(trailing, on=account_col, how="left"))

    # Fill NaNs
    for c in ["Latest_Month_Analyzed_Charges",
              "Latest_Month_Combined_Result",
              "Trailing_12M_Analyzed_Charges",
              "Trailing_12M_Combined_Result",
              "Latest_Month_ECR",
              "Trailing_12M_Avg_ECR"]:
        if c in out.columns:
            out[c] = out[c].fillna(0.0)

    # Order
    cols = [
        account_col,
        "Latest_Month_Analyzed_Charges",
        "Latest_Month_Combined_Result",
        "Trailing_12M_Analyzed_Charges",
        "Trailing_12M_Combined_Result",
        "Latest_Month_ECR",
        "Trailing_12M_Avg_ECR",
        "Primary_Officer_Name_XAA",
        "Secondary_Officer_Name_XAA",
        "Treasury_Officer_Name_XAA",
    ]
    return out[[c for c in cols if c in out.columns]].copy()

# -------- Use the summarizer (defaults to previous month-end relative to today) --------
summarized_xaa = summarize_xaa_for_latest_and_ttm(
    xaa_data,
    date_col='Cycle End Date'
    # target_month="2025-07",  # <- optionally pass to force August 2025 (matches 2025-08-31)
)

# Enforce schema for officer name columns
summarized_xaa_schema = {
    'Primary_Officer_Name_XAA': 'str',
    'Secondary_Officer_Name_XAA': 'str',
    'Treasury_Officer_Name_XAA': 'str'
}
summarized_xaa = cdutils.input_cleansing.enforce_schema(summarized_xaa, summarized_xaa_schema)

# Prepare for merge
summarized_xaa = summarized_xaa.rename(columns={'Debit Account Number': 'acctnbr'}).copy()
assert summarized_xaa['acctnbr'].is_unique, "Duplicates in summarized XAA per account."



In [ ]:
# Merge with main data
merged_data = pd.merge(data, summarized_xaa, on='acctnbr', how='left')

# Fill numeric NaNs post-merge
for item in [
    'Latest_Month_Analyzed_Charges',
    'Latest_Month_Combined_Result',
    'Trailing_12M_Analyzed_Charges',
    'Trailing_12M_Combined_Result',
    'Latest_Month_ECR',
    'Trailing_12M_Avg_ECR',
]:
    if item in merged_data.columns:
        merged_data[item] = merged_data[item].fillna(0.0)

# Sort descending order of notebal
if 'notebal' in merged_data.columns:
    merged_data = merged_data.sort_values(by='notebal', ascending=False)

# %%

# Downstream core transform
formatted_data = src.core_transform.main_pipeline(merged_data)

# Standardize friendly column names
formatted_data = formatted_data.rename(columns={
    'portfolio_key': 'Portfolio Key',
    'product': 'Product',
    '3Mo_AvgBal': '3Mo Avg Bal',
    'TTM_AvgBal': 'TTM Avg Bal',
    'TTM_DAYS_OVERDRAWN': 'TTM Days Overdrawn',
    'TTM_NSF_COUNT': 'TTM NSF Count'
}).copy()

# Create summary sheet
summary_data = formatted_data[~(formatted_data['Portfolio Key'] == "") & (formatted_data['Acct No.'] == "")].copy()
summary_data = summary_data[[
    'Portfolio Key',
    'Borrower Name',
    'Account Officer',
    'Cash Management Officer',
    'Current Balance',
    'Interest Rate',
    '3Mo Avg Bal',
    'TTM Avg Bal',
    'Year Ago Balance',
    'TTM Days Overdrawn',
    'TTM NSF Count',
    'Current Mo Analyzed Fees (Pre-ECR)',
    'Current Mo Net Analyzed Fees (Post-ECR)',
    'TTM Analyzed Fees (Pre-ECR)',
    'TTM Net Analyzed Fees (Post-ECR)',
    'Current ECR'
]].copy()

# Output to excel
OUTPUT_DIR = Path('./output')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH = OUTPUT_DIR / "business_deposits_concentration_with_xaa.xlsx"
with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl") as writer:
    formatted_data.to_excel(writer, sheet_name='Relationship Detail', index=False)
    summary_data.to_excel(writer, sheet_name='Relationship Summary', index=False)
    merged_data.to_excel(writer, sheet_name='Unformatted', index=False)

# Format excel
src.output_to_excel_multiple_sheets.format_excel_file(OUTPUT_PATH)



In [ ]:
formatted_data

In [ ]:
summary_data

In [ ]:
final_report_post_ecr = summary_data['Current Mo Net Analyzed Fees (Post-ECR)'].sum()
final_report_post_ecr

In [ ]:
xaa_data['Cycle End Date'] = pd.to_datetime(xaa_data['Cycle End Date'])
xaa_data_rct = xaa_data[xaa_data['Cycle End Date'] == datetime(2025, 8, 31)].copy()
xaa_total = xaa_data_rct['Combined Result for Settlement Period'].sum()
xaa_total

In [ ]:
difference = xaa_total - final_report_post_ecr
print(f"XAA total: ${xaa_total:,.2f}")
print(f"Final total: ${final_report_post_ecr:,.2f}")
print(f"Difference between XAA & Final Report for Current Month Post-ECR:")
print(f"${difference:,.2f}")

In [ ]:
xaa_data

In [ ]:
# xaa_only_total = 563.41
# print(f"${final_report_post_ecr + xaa_only_total:,.2f}")


In [ ]:
# Merge with main data
merged_data2 = pd.merge(data, summarized_xaa, on='acctnbr', how='outer', indicator=True)

In [ ]:
# Merge with main data
# merged_data2 = pd.merge(merged_data, summarized_xaa, on='acctnbr', how='outer', indicator=True)
# Filter rows based on column: '_merge'
merged_data2 = merged_data2[merged_data2['_merge'].apply(str).str.contains("right", regex=False, na=False, case=False)]

In [ ]:
merged_data2['acctnbr'].nunique()

In [ ]:
merged_data2

In [ ]:
merged_data2.info()

In [ ]:
merged_data2['Latest_Month_Combined_Result'].sum()

In [ ]:
merged_data2 = merged_data2[[
    'acctnbr',
    'Latest_Month_Combined_Result',
    'Primary_Officer_Name_XAA',
    'Secondary_Officer_Name_XAA',
    'Treasury_Officer_Name_XAA'
]].copy()

In [ ]:
# Get more data about missing XAA items

In [ ]:
merged_data2

In [ ]:
# Lakehouse
LAKEHOUSE_PATH = Path(r"C:\Users\w322800\Documents\lakehouse") 

# Bronze
BRONZE = LAKEHOUSE_PATH / "bronze"

# Silver
SILVER = LAKEHOUSE_PATH / "silver"

# Gold
GOLD = LAKEHOUSE_PATH / "gold"

In [ ]:
from deltalake import DeltaTable
full_data = DeltaTable(BRONZE / "wh_acctcommon").to_pandas()

In [ ]:
full_data

In [ ]:
full_data.info()

In [ ]:
full_data = full_data[[
    'acctnbr',
    'ownersortname',
    'product',
    'currmiaccttypcd',
    'curracctstatcd',
    'acctofficer',
    'contractdate',
    'closedate',
    'notebal'
]].copy()

In [ ]:
full_data['acctnbr'] = full_data['acctnbr'].astype(str)

In [ ]:
xaa_no_match = merged_data2.merge(full_data, how='left', on='acctnbr')

In [ ]:
xaa_no_match

In [ ]:
xaa_no_match['Latest_Month_Combined_Result'].sum()

In [ ]:
import numpy as np
xaa_no_match['Active/Inactive'] = np.where(xaa_no_match['curracctstatcd'].isin(['ACT','DORM']), 'Active','Inactive')

In [ ]:
xaa_no_match

In [ ]:
xaa_no_match.info()

In [ ]:
xaa_no_match = xaa_no_match[[
    'acctnbr',
    'Active/Inactive',
    'Latest_Month_Combined_Result',
    'ownersortname',
    'product',
    'contractdate',
    'closedate',
    'notebal'
]].copy()

In [ ]:
xaa_no_match

In [ ]:
xaa_no_match = xaa_no_match.sort_values(by='Latest_Month_Combined_Result', ascending=False).copy()

In [ ]:
OUTPUT_PATH = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Commercial Lending\Business_Concentration_of_Deposits\Production\output") / "validation_no_match.xlsx"
xaa_no_match.to_excel(OUTPUT_PATH, sheet_name='Sheet1',index=False)